In [ ]:
!nvidia-smi

Sun Apr 28 18:00:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
# %pip install cupy-cuda11x

In [ ]:
import numpy as np
import cupy as cp

We have copy and pasted the functions defined in our pythontopython2.py below:

In [ ]:
def setup(file_name):
    file = open(file_name + ".py", "a")
    file.write("import cupy")

def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name
    return var

def numpy_add_to_py(a, b):
    a_name = get_var_name(a)
    b_name = get_var_name(b)
    return f"cupy.add({a_name}, {b_name})"

def numpy_subtract_to_py(a, b):
    a_name = get_var_name(a)
    b_name = get_var_name(b)
    return f"cupy.subtract({a_name}, {b_name})"

def numpy_sum_to_py(a):
    a_name = get_var_name(a)
    return f"cupy.sum({a_name})"

def numpy_dot_to_py(a, b):
    a_name = get_var_name(a)
    b_name = get_var_name(b)
    return f"cupy.dot({a_name}, {b_name})"

def array_to_py(variable_name, list_values):
    # list_values is a string containing comma-separated values
    # Turn list_values into a list, and each value in the list into an integer
    values = [int(value) for value in list_values.split(',')]

    return f"{variable_name} = cupy.array({values})"

We want to show firstly that these functions produce strings as expected.

In [ ]:
setup('example_script')
# Expected to create or append to 'example_script.py' with 'import cupy'

In [ ]:
x = np.array([10, 20, 30])
print(get_var_name(x))  # Should print 'x' if 'x' is in the global namespace

x


In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(numpy_add_to_py(a, b))  # Expected output: "cupy.add(a, b)"

cupy.add(a, b)


In [ ]:
a = np.array([7, 8, 9])
b = np.array([1, 2, 3])
print(numpy_subtract_to_py(a, b))  # Expected output: "cupy.subtract(a, b)"

cupy.subtract(a, b)


In [ ]:
a = np.array([10, 20, 30])
print(numpy_sum_to_py(a))  # Expected output: "cupy.sum(a)"

cupy.sum(a)


In [ ]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
print(numpy_dot_to_py(a, b))  # Expected output: "cupy.add(a, b)"

cupy.dot(a, b)


In [ ]:
variable_name = 'data_array'
list_values = '100,200,300,400,500'
print(array_to_py(variable_name, list_values))  # Expected output: "data_array = cupy.array([100, 200, 300, 400, 500])"

data_array = cupy.array([100, 200, 300, 400, 500])


The next step would be to test the output strings actually run when they are  written to a file. In the next cell, we have copied all results from previous steps to emulate the file produced. We also want to compare the numpy vs. cupy run time.

In [ ]:
import time

In [24]:
import numpy as np
import cupy as cp
import time

array_size = 10000000  # 10 million elements

a_gpu = cp.random.rand(array_size)
b_gpu = cp.random.rand(array_size)

# creating large arrays for NumPy
a_cpu = np.random.rand(array_size)
b_cpu = np.random.rand(array_size)

# timing CuPy operations
start_gpu = time.time()
cp.add(a_gpu, b_gpu)
cp.subtract(a_gpu, b_gpu)
cp.sum(a_gpu)
time_gpu = time.time() - start_gpu

# timing NumPy operations
start_cpu = time.time()
np.add(a_cpu, b_cpu)
np.subtract(a_cpu, b_cpu)
np.sum(a_cpu)
time_cpu = time.time() - start_cpu

print("CuPy execution time: {:.5f} seconds".format(time_gpu))
print("NumPy execution time: {:.5f} seconds".format(time_cpu))


CuPy execution time: 0.00103 seconds
NumPy execution time: 0.06860 seconds
